### 自动求导

假设我们想对$y=2x^Tx$关于列向量x求导

In [12]:
import torch  
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算$y$关于$x$的梯度之前，我们需要一个地方来储存梯度。

In [13]:
x.requires_grad_(True)  # 等价于`x = torch.arange(4.0, requires_grad=True)
x.grad  # 默认值是None

现在计算$y$

In [14]:
y = 2 * torch.dot(x, x)  
y

tensor(28., grad_fn=<MulBackward0>)

调用反向传播函数来自动计算$y$关于$x$每个分量的梯度

In [15]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [16]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算$x$的另一个函数

In [24]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的梯度值  
x.grad.zero_()  # _表示重写内容，这里表示把0写入梯度  
y = x.sum()  
y.backward()  # 向量求sum之后梯度全为1
x.grad, y, print(x.sum())

tensor(6., grad_fn=<SumBackward0>)


(tensor([1., 1., 1., 1.]), tensor(6., grad_fn=<SumBackward0>), None)

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。

In [23]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数  
x.grad.zero_()
y = x * x  
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [25]:
x.grad.zero_()
y = x * x  
u = y.detach()  # 把y当作一个常数而不是关于x的函数
z = u * x  

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [26]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x  

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流(例如:条件、循环或任意函数调用)，我们任然可以计算得到变量的梯度。

In [38]:
def f(a):
    b = a * 2  
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b 
    else:
        c = 100 * b  
        return c  

a = torch.randn(size=(), requires_grad=True)  # a是一个随机数，其中size为空表示一个标量，需要梯度
d = f(a)
d.backward()

a.grad == d / a

tensor(True)